# About: Moodleコンテナの起動

---

Moodleコンテナを起動します。

## 全体構成

![構成](images/moodle-021-01.png)

構築するMoodle環境は次の２つのDockerコンテナで構成されます。

* [Moodle](https://harbor.vcloud.nii.ac.jp/harbor/projects/2/repositories/vcp%2Fmoodle-simple)
  - MoodleのPHPスクリプトを実行するコンテナ
* [MariaDB](https://hub.docker.com/_/mariadb/)
  - Moodleの設定を保存するデータベース
  
構築環境となるノードを事前に「011-VCノードの作成」、「012-EC2インスタンスの作成」、「013-Azure仮想マシンの作成」のいずれかのNotebookで作成しておく必要があります。

## 準備

### グループ名の指定

Moodle環境の構築対象を指定するために Ansible のグループ名を指定します。

ノード作成をどのNotebookで行ったかによって、ここで指定する値は異なります。各Notebookでどの値がAnsibleのグループ名に対応するのかを以下に示します。

* 011-VCノードの作成.ipynb
    - UnitGroup名: `ugroup_name`
* 012-EC2インスタンスの作成.ipynb
    - EC2インスタンスの名前: `aws_ec2_name`
* 013-Azure仮想マシンの作成.ipynb
    - 仮想マシンの名前: `azure_vm_name`

ノードを作成した時に指定した値を確認するために `group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

### 前提条件の確認

このNotebookを実行するための前提条件を満たしていることを確認します。

前提となる条件を以下に示します。

* Ansibleから操作可能であること
* Ansibleから管理者権限でコマンドを実行できること
* Docker が利用可能であること
* docker compose コマンドが利用可能であること
* RockyLinuxであること

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    raise RuntimeError(f"ERROR: not exists {target_group}")

対象となるホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

管理者権限でコマンド実行可能であることを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

Dockerが利用可能であることを確認します。

In [ ]:
!ansible {target_group} -a 'docker info'

docker composeコマンドが利用可能であることを確認します。

In [ ]:
!ansible {target_group} -a 'docker compose version'

構築対象のノードが RockyLinux 8 であること確認します。

In [ ]:
import json

out = !ansible {target_group} -m setup
try:
    idx = [i for i, x in enumerate(out) if x.endswith('| SUCCESS => {')][0]
    data = json.loads(' '.join(['{'] + out[(idx + 1):]))
    distribution = data['ansible_facts']['ansible_distribution']
    distribution_major_version = data['ansible_facts']['ansible_distribution_major_version']
    if distribution != 'Rocky':
        raise RuntimeError(f"ERROR: {distribution} {distribution_major_version}")
except:
    for line in out:
        print(line)
    raise RuntimeError("error!")

## パラメータ設定

### Moodle
Moodleに関するパラメータを指定します。

![Moodle設定](images/moodle-021-02.png)

#### Moodleのコンテナイメージ

Moodleコンテナのイメージ名を指定してください。

ここで指定する値は `harbor.vcloud.nii.ac.jp` にあらかじめ用意してある Moodle イメージから選択する必要があります。次のセルを実行すると選択可能な値の一覧を表示します。

In [ ]:
import itertools
import math
import re
import requests
%run scripts/utils.py

url = 'https://harbor.vcloud.nii.ac.jp/api/v2.0/projects/vcp/repositories/moodle-simple'
res = requests.get(url)
page_size = 100
artifact_count = res.json()["artifact_count"]
tags = []
for idx in range(math.ceil(artifact_count / page_size)):
    res = requests.get(f"{url}/artifacts?page={idx + 1}&page_size={page_size}")
    tags.extend([[tag["name"] for tag in x["tags"]] for x in res.json()])
tags = sorted([
    f"harbor.vcloud.nii.ac.jp/vcp/moodle-simple:{x}"
    for x in itertools.chain.from_iterable(tags)
    if not x.endswith('-ssl') and re.match(r'^\d', x) and check_version(x)
], reverse=True)
for tag in tags:
    print(tag)

コンテナイメージのタグによって、どのリリースに対応するMoodle環境を構築するかを指定できます。`4.x.x` のようなタグはマイナーリリースまで特定したコンテナイメージとなります。また `4.x` のようなタグは各ブランチにおける最新のマイナーリリースを意味しています。Moodleのリリース状況に関しては[Releases | Moodle Developer Resources](https://moodledev.io/general/releases)を参照してください。現在の最新のLTS(Long Term Support)はMoodle 4.5となっています。

In [ ]:
# (例)
# moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle-simple:4.5.8'
# moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle-simple:4.5'

moodle_image_name = 

#### Moodleの管理者ユーザ名

Moodleの管理者ユーザ名を指定します。

> Moodleのユーザ名に指定できる文字は、小文字英数字と`_`, `-`, `@`, `.`です。

In [ ]:
# (例)
# moodle_admin_name = 'admin'

moodle_admin_name = 

#### Moodleの管理者パスワード

管理者パスワードを指定します。

次のセルを実行するとパスワード入力用の枠が表示されます。管理者パスワードを入力してください（入力後に Enter キーを押すことで入力が完了します）。

> パスワードの値は `admin` 以外の値を指定してください。

In [ ]:
from getpass import getpass
moodle_admin_password = getpass()

#### MoodleのURL

MoodleではサイトのURLを`config.php`の`$CFG->wwwroot`に設定する必要があります。構築対象のURLを指定してください。

In [ ]:
# (例)
# moodle_url = 'http://moodle.example.org'
# moodle_url = 'http://172.30.2.100'
# moodle_url = 'https://moodle.example.org'

moodle_url = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。ただしパスワードなどの秘匿情報についてはファイルへの保存を行いません。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py
check_parameters(
    'moodle_admin_name',
    'moodle_url',
    nb_vars=locals(),
)

パラメータの値を group_vars ファイルに保存します。

In [ ]:
%run scripts/group.py

update_group_vars(
    target_group,
    _file="20-moodle.yml",
    moodle_image_name=moodle_image_name,
    moodle_url=moodle_url,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/20-moodle.yml

### データベース

Moodleの設定値などを保存するデータベースに関するパラメータを指定します。

![データベース設定](images/moodle-021-03.png)

#### データベースのコンテナイメージ


データベースコンテナのイメージ名を指定してください。このNotebookが構築する環境では MariaDBのコンテナイメージを指定することができます。

In [ ]:
db_image_name = 'mariadb:11.8'

Moodleの設定ファイル`config.php`の中で `$CFG->dbtype` に指定するデータベースの種別を指定してください。

> このNotebookでは `mariadb` を指定した場合の動作確認のみ行っています。

In [ ]:
# (例)
# db_type = 'mariadb'    # MariaDB
# db_type = 'mysql'      # MySQL

db_type = 'mariadb'

#### データベース名

Moodleが使用するデータベース名を指定してください。

In [ ]:
# (例)
# db_moodle_db = 'moodle'

db_moodle_db = 'moodle'

#### データベースの接続ユーザ

Moodleのデータベースに接続するためのデータベースのユーザ名を指定してください。

In [ ]:
# (例)
# db_moodle_db_user = 'moodle'

db_moodle_db_user = 'moodle'

#### データベースのパスワード


Moodleのデータベースに接続するためのパスワードを指定します。

In [ ]:
from getpass import getpass
db_moodle_db_password = getpass()

#### データベースの管理者パスワード

管理者のパスワードを指定します。

In [ ]:
db_root_password = getpass()

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。ただしパスワードなどの秘匿情報についてはファイルへの保存を行いません。

In [ ]:
update_group_vars(
    target_group,
    _file="20-moodle.yml",
    db_image_name=db_image_name,
    db_type=db_type,
    db_moodle_db=db_moodle_db,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/20-moodle.yml

## 設定ファイルの配置

Moodleコンテナ、DBコンテナを実行するのに必要となる設定ファイルを構築環境に配置します。

![設定ファイルの配置](images/moodle-021-04.png)

### compose.yaml の配置

このNotebookで構築するMoodle環境は複数のコンテナから構成されています。複数コンテナの管理を容易にするために`docker compose`を利用します。`docker compose`を利用することで、YAMLで記述した設定ファイル`compose.yaml`に記述したコンテナ構成に従い複数のコンテナの起動、停止などの操作が行えます。

まず構築環境に`compose.yaml`などを格納するディレクトリ`/srv/moodle`を作成します。

In [ ]:
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle state=directory owner={{{{ansible_ssh_user}}}}'

作成したディレクトリに `compose.yaml` を配置します。

In [ ]:
!ansible {target_group} -Dv -m template \
    -a 'src=template/compose.yaml.j2 dest=/srv/moodle/compose.yaml backup=yes'

### 秘匿情報に関する値を構築環境に配置する

この構築環境ではMoodleの管理者パスワードやデータベースの接続パスワード、ユーザ名などの秘匿情報を[docker secrets](https://docs.docker.com/engine/swarm/secrets/)の機能を用いて管理します。docker secretsの設定は`compose.yaml`の[secrets](https://docs.docker.com/reference/compose-file/secrets/)として記述しますが、各パラメータの値は構築環境の個別のファイルに格納することになります。各パラメータとその値を格納するファイル名の対応を下表に示します。

|パラメータ|ファイル名|
|:---|:---|
|Moodleの管理者ユーザ名|/srv/moodle/secrets/MOODLE_ADMIN|
|Moodleの管理者パスワード|/srv/moodle/secrets/MOODLE_ADMIN_PASSWORD|
|データベース名|/srv/moodle/secrets/DB_DATABASE|
|データベースの接続ユーザ名|/srv/moodle/secrets/DB_USER|
|データベースの接続パスワード|/srv/moodle/secrets/DB_PASSWORD|
|データベースの管理者パスワード|/srv/moodle/secrets/DB_ROOT_PASSWORD|

秘匿情報の値を格納したファイルを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

!ansible {target_group} -b -m file -a \
    'path=/srv/moodle state=directory owner={{{{ansible_ssh_user}}}}'
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle/secrets state=directory owner={{{{ansible_ssh_user}}}}'
params = {
    'MOODLE_ADMIN': moodle_admin_name,
    'MOODLE_ADMIN_PASSWORD': moodle_admin_password,
    'DB_DATABASE': db_moodle_db,
    'DB_USER': db_moodle_db_user,
    'DB_PASSWORD': db_moodle_db_password,
    'DB_ROOT_PASSWORD': db_root_password,
}

with TemporaryDirectory() as workdir:
    for key, value in params.items():
        var_path = Path(workdir) / key
        with var_path.open(mode='w') as f:
            f.write(value)
        !ansible {target_group} -b -m copy \
            -a 'src={var_path} dest=/srv/moodle/secrets/ mode=0400 owner=root'

作成したファイルの一覧を確認します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    tree secrets'

### 各コンテナの設定ファイルを配置する

このNotebookで構築する環境では、コンテナ内で実行する Apache HTTP server の`httpd.conf`などの設定ファイルを構築環境のホスト側に配置しています。これは各コンテナイメージに状態を持たせないようにして、バックアップ、リストアなどを容易に行うために行っています。ホスト側に配置した設定ファイルは[bind mount](https://docs.docker.com/storage/bind-mounts/)の機能を利用してコンテナから参照できるようにしています。

bind mountによって配置する設定ファイルのホスト環境におけるパスとコンテナ環境におけるパスの対応関係の主なものを以下に示します。

|コンテナ名|コンテナ環境のパス|ホスト環境のパス|
|:---|:---|:---|
|moodle|/etc/httpd/conf/httpd.conf|/srv/moodle/moodle/conf/httpd/conf/httpd.conf|
|moodle|/etc/httpd/conf.d/|/srv/moodle/moodle/conf/httpd/conf.d/|
|moodle|/etc/httpd/conf.modules.d/|/srv/moodle/moodle/conf/httpd/conf.modules.d/|
|moodle|/etc/php.ini|/srv/moodle/moodle/conf/php.ini|
|moodle|/etc/php.d/|/srv/moodle/moodle/conf/php.d/|
|moodle|/etc/php-fpm.conf|/srv/moodle/moodle/conf/php-fpm.conf|
|moodle|/etc/php-fpm.d/|/srv/moodle/moodle/conf/php-fpm.d/|
|db|/etc/mysql/mariadb.cnf|/srv/moodle/db/conf/mariadb.cnf|
|db|/etc/mysql/mariadb.conf.d/|/srv/moodle/db/conf/mariadb.conf.d/|

この節では、コンテナイメージの設定ファイルをコピーして構築環境のホスト側への配置を行います。

コンテナ内の設定ファイルをホスト側にコピーするために仮のコンテナを起動します。仮のコンテナを起動するために`compose.setup-*.yaml`を用います。

実際の操作を行う前にチェックモードで ansible を実行します。

In [ ]:
!ansible-playbook -l {target_group} -CDv playbooks/deploy-setup-conf.yml || true

実際に設定ファイルの取得を行います。

In [ ]:
!ansible-playbook -l {target_group} -v playbooks/deploy-setup-conf.yml

ホスト環境にコピーした設定ファイルを確認します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    tree -L 3 db moodle'

### ログ関連の準備

コンテナで実行するサービスのログに関するセットアップを行います。

#### ログ出力先のディレクトリの作成

このNotebookで構築する環境では、コンテナ内で実行する Apache HTTP server などのログ出力先ディレクトリをホスト側のディレクトリに[bind mount](https://docs.docker.com/storage/bind-mounts/)します。これによりホスト側からもログの参照が容易になります。

ログ出力先ディレクトリのコンテナ環境とホスト環境での対応関係を以下に示します。

|コンテナ名|コンテナ環境のパス|ホスト環境のパス|
|:---|:---|:---|
|moodle|/var/log/httpd|/var/log/httpd|
|db|/var/log/mysql|/var/log/mysql|

ログ出力先のディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a 'path=/var/log/httpd state=directory'
!ansible {target_group} -b -m file -a 'path=/var/log/mysql owner=999 group=adm state=directory'

#### logrotateの設定ファイルを配置する

Moodleコンテナ、DBコンテナのログをローテーションするための設定ファイルを配置します。logrotateはホスト環境で実行するので、ホスト側の `/etc/logrotate.d/` に配置します。 

In [ ]:
!ansible {target_group} -b -m copy -a 'src=template/logrotate.d/httpd dest=/etc/logrotate.d/'
!ansible {target_group} -b -m copy -a 'src=template/logrotate.d/mysql-server dest=/etc/logrotate.d/'

#### mysqladmin の設定ファイルを配置する

MariaDB/MySQL に対して logrotate を行うには管理者権限で `mysqladmin flush-logs` などのコマンドを実行出来るようにする必要があります。これを可能にするために管理者パスワードの情報を `/root/.my.cnf` に格納します。

コンテナに状態を持たせないようにするために設定ファイルはホスト環境に配置したものを bind mountすることにします。コンテナ環境、ホスト環境におけるパスを以下に示します。

|コンテナ環境のパス|ホスト環境のパス|
|:---|:---|
|/root/.my.cnf|/srv/moodle/db/misc/my.cnf|

設定ファイルをホスト側に配置します。

In [ ]:
import yaml

with TemporaryDirectory() as workdir:
    vars_path = Path(workdir) / 'moodle.yml'
    with vars_path.open(mode='w') as f:
        yaml.safe_dump({'db_root_password': db_root_password}, f)
    !ansible {target_group} -b -m template -e @{vars_path} \
        -a 'src=template/mysql/my.cnf.j2 dest=/srv/moodle/db/misc/my.cnf \
        mode=0600 owner=root group=root'

## アプリケーションコンテナの起動

Moodleコンテナ、データベースコンテナを起動して、Moodle環境を起動します。

![コンテナの起動](images/moodle-021-05.png)

コンテナの起動を行います。Moodleのサービスが開始されるまで数分程度を要します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up --wait --remove-orphans'

## Moodle を利用できることを確認

構築したMoodle環境にアクセスし、Moodle が利用できることを確認します。

### アドレスの確認

次のセルを実行すると表示されるリンクが、構築したMoodle環境のアドレスです。リンクをクリックしてMoodle環境にログインしてください。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}" target="_blank">{0}</a>'.format(moodle_url))

### ライブログの確認

Moodle のライブログを確認します。次のセルを実行すると表示されるリンクをクリックしてください。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/report/loglive/index.php" target="_blank">{0}/report/loglive/index.php</a>'.format(moodle_url))

## コンテナのログを確認する

MoodleコンテナのApache HTTP Serverのログはホスト環境の `/var/log/httpd/` に出力されます。アクセスログ `access_log` の内容を確認してみます。

In [ ]:
!ansible {target_group} -a 'tail /var/log/httpd/access_log'

DBコンテナのログはホスト環境の `/var/log/mysql/` に出力されます。エラーログ `error.log` の内容を確認してみます。

In [ ]:
!ansible {target_group} -b -a 'tail /var/log/mysql/error.log'

## crontabの設定

Moodleのスケジュールタスクを実行するために必要となるスクリプトを1分毎に実行するように crontab を設定します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec moodle \
    /usr/bin/php /var/www/moodle/admin/cli/cron.php --enable'
!ansible {target_group} -m cron -a 'name=PATH env=yes job=/usr/bin:/usr/local/bin'
!ansible {target_group} -m cron -a 'name={target_group} \
    job="docker compose --project-directory=/srv/moodle exec -T moodle \
    /usr/bin/php /var/www/moodle/admin/cli/cron.php > /dev/null"'